# Basics of Pandas for RDS Consultation

Most patrons who come asking for Python help are asking for help with manipulating data in Pandas. Pandas is the standard in analysis and visualization in data science, so it's a useful library to know. This is just a very basic introduction, however, and it relies, largely, on the [official Pandas cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf).

First, you need to import the `numpy` and `pandas` libraries.

In [ ]:
import numpy as np
import pandas as pd

## Read in Data

Pandas, now available as `pd` has several methods for reading in tabular data. In addition to `pd.read_csv()`, there's also other `.read_*()` methods for Excel spreadsheets, html, and so on. As always, hit `tab` while typing a method name to see what other methods there are with a similar name.

For this cheatsheet, we'll be using the 2019 [ZiPS projections](http://m.mlb.com/glossary/projection-systems/szymborski-projection-system) for batters in Major League Baseball. The data is available [from Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips) by following the link and then clicking on "Export Data" on the right side of the page. A `.csv` should download. Now, let's read it in and look at the first five rows.

In [ ]:
batters = pd.read_csv("Fangraphs Leaderboard.csv")
batters.head()

This is a reasonable mix of scalar and numerical data. But there's an immediate hiccup: in reading the `.csv`, Pandas immediately assigns an id to each row, but the data already _has_ a unique identifier for each player, the "playerid" row. So, as a first step in cleaning, let's reshape the data so that "playerid" is the id field:

In [ ]:
batters.set_index("playerid", inplace=True)
batters.head()

## Filter, Map, Reduce

From a programming background, the three atomic actions I can make with a dataset are filter, map, and reduce. Filter shrinks the number of rows, map changes what each row represents, and reduce tells me something about the dataset as a whole. It's very tempting to think of a Pandas dataframe as just an array of arrays and use iteration to accomplish these tasks, but Pandas exposes a built in syntax to simplify the tasks.

### Filter

For example, Pandas lets you filter the dataframe based on a truth test you write inside brackets, like this: `dataframe[truth test]`. So, for every batter who is proejcted to hit more than 35 home runs ("HR"), we can simply do:

In [ ]:
batters[batters.HR > 35]

Note that we can access the column name with dot notation. We can also use the column name in brackets. `batters[batters["HR"] > 35]` gives the same response.

Unfortunately, this does not work for truth tests involving strings, as a scalar value in Pandas is not, strictly speaking, a string:

In [ ]:
batters.Name.dtype

As an object, that means we need to first change the values to strings in order to make the truth test work. Furthermore, the standard Python `"value" in string` truth test is ambiguous in Pandas, requiring the use of a `.contains()` method. So to find every player named "Dustin," for example:

In [ ]:
batters[batters.Name.str.contains("Dustin")]

The `.isna()` method can shake out empty values in certain columns, as well:

In [ ]:
batters[batters.Team.isna()]

## Map

Mapping is the process of effecting some sort of change on each row of a dataset and yielding a dataset on the other side of the process that has the same number of rows. One way to use mapping is by calculating new columns based on pre-existing columns. For example, the ZiPS projections do not include total bases as a projected statistic. However, we can calculate total bases from the number of hits, doubles, and homeruns:

In [ ]:
batters["TB"] = batters.H + batters["2B"] + (2 * batters["3B"]) + (3 * batters.HR)
batters[batters.TB > 300]

### Reduce

Reduction gives an image of the dataset as a whole, by perhaps summing values, finding their averages, and so on. Its descriptive power is indicated by the method `.describe()`:

In [ ]:
batters.describe()

The `.sum()`, `.mean()` and similar methods can be used on a column-by-column basis, as well. 
Another way of thinking of reducing is similar to the pivot table in Excel, where data are aggregated based on groupings. In Pandas, that's the `.groupby()` method. To see how many runs each team is projected to score, for example, it's possible to group by team name, sum the values, and then sort by descending values:

In [ ]:
batters.groupby("Team").sum().sort_values("R", ascending=False)

## Loc Like a Pro

The `.loc[]` method lets you slice and dice a dataframe. It anticipates a matrix taking the form of `df.loc[rows, columns]`, but `rows` can be a truth test, a list of specific ids, or a slice (using `:`). The same holds true for `columns`: 

In [ ]:
batters.loc["20123"] # Give me the row with the id of "20123" as a Series
batters.loc[:, "H"] # Give me all the rows (`:`) and then the "H" column. Eq to "batters.H"
batters.loc[:, "H":"HR"] # All rows, with columns from "H" to "HR" inclusive.
batters.loc[batters.RBI > 100, "H":"HR"] # As above, but limit rows to those where RBI > 100
batters.loc[batters.RBI > 100, ["Name", "RBI", "H", "2B", "3B", "HR"]] # Add Name & RBI for clarity


The `.iloc[]` method performs similarly, but it uses (integer) positions instead of labels.